In [2]:
import pandas as pd

df = pd.read_csv("/home/kush/machine_learning/ClassificationThyroidDiseases_ByUltrasoundImage/data/total_data.csv")
df.head()

,name,default_mean,default_std,default_min,default_max,blur_mean,blur_std,blur_min,blur_max,sobel_mean,...,laplacian_std,canny_mean,canny_std,figure_mean,figure_area,figure_max,figure_min,figure_std,figure_perimeter,target
0,69_2,88.001217,38.134743,13,255,87.993780,35.513409,19,253,27.873759,...,123.031301,10.581076,50.854845,72.027110,2287.0,183.0,18.0,23.668996,180.076865,0
1,380_1,82.335703,41.009582,7,255,82.334771,37.857679,15,231,26.857104,...,122.794972,17.555377,64.563379,71.692199,57235.0,249.0,9.0,32.342588,913.065723,0
2,158_2,97.597160,43.821470,7,255,97.597751,40.915805,25,248,28.786973,...,122.727027,14.607444,59.258086,75.991837,6248.0,216.0,29.0,24.794651,299.780540,0
3,66_1,78.756686,31.397057,9,245,78.757682,27.661744,25,202,25.348055,...,123.245893,17.505292,64.478014,63.814262,12523.0,173.0,11.0,22.628529,557.980631,0
4,69_3,93.166859,37.735053,12,255,93.159992,34.687413,22,247,28.895076,...,123.075315,9.070826,47.231141,82.739580,2903.0,189.0,19.0,23.651310,210.073542,0


In [3]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["name", "target"])
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [4]:
y_train.value_counts()

target
1    144
0     81
Name: count, dtype: int64

In [6]:
y_test.value_counts()

target
1    49
0    27
Name: count, dtype: int64

In [13]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

models = {
    "SVM": SVC(kernel="rbf", random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "LogisticRegression": LogisticRegression(random_state=42),
    "RandomForest": RandomForestClassifier(max_depth=5, random_state=42),
    "DecisionTree": DecisionTreeClassifier(max_depth=5, random_state=42),
    "Xgboost": xgb.XGBClassifier(objective="binary:logistic"),
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(name)
    print("Accuracy", accuracy_score(y_test, y_pred))
    print("Precision", precision_score(y_test, y_pred))
    print("Recall", recall_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

SVM
Accuracy 0.6578947368421053
Precision 0.6533333333333333
Recall 1.0
              precision    recall  f1-score   support

           0       1.00      0.04      0.07        27
           1       0.65      1.00      0.79        49

    accuracy                           0.66        76
   macro avg       0.83      0.52      0.43        76
weighted avg       0.78      0.66      0.53        76

[[ 1 26]
 [ 0 49]]
KNN
Accuracy 0.6052631578947368
Precision 0.7111111111111111
Recall 0.6530612244897959
              precision    recall  f1-score   support

           0       0.45      0.52      0.48        27
           1       0.71      0.65      0.68        49

    accuracy                           0.61        76
   macro avg       0.58      0.59      0.58        76
weighted avg       0.62      0.61      0.61        76

[[14 13]
 [17 32]]
LogisticRegression
Accuracy 0.5657894736842105
Precision 0.6290322580645161
Recall 0.7959183673469388
              precision    recall  f1-score   s

/home/kush/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.57      0.30      0.39        27
           1       0.69      0.88      0.77        49

    accuracy                           0.67        76
   macro avg       0.63      0.59      0.58        76
weighted avg       0.65      0.67      0.64        76

[[ 8 19]
 [ 6 43]]
DecisionTree
Accuracy 0.618421052631579
Precision 0.6923076923076923
Recall 0.7346938775510204
              precision    recall  f1-score   support

           0       0.46      0.41      0.43        27
           1       0.69      0.73      0.71        49

    accuracy                           0.62        76
   macro avg       0.58      0.57      0.57        76
weighted avg       0.61      0.62      0.61        76

[[11 16]
 [13 36]]
Xgboost
Accuracy 0.5657894736842105
Precision 0.6428571428571429
Recall 0.7346938775510204
              precision    recall  f1-score   support

           0       0.35      0.26      0.30        27
           1 

In [ ]:
from collections import Counter

from imblearn.over_sampling import SMOTE

print("Распределение классов до SMOTE:", Counter(y_train))

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Распределение классов после SMOTE:", Counter(y_train_resampled))

Распределение классов до SMOTE: Counter({1: 144, 0: 81})
Распределение классов после SMOTE: Counter({1: 144, 0: 144})


In [11]:
for name, model in models.items():
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)
    print(name)
    print("Accuracy", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

SVM
Accuracy 0.5131578947368421
              precision    recall  f1-score   support

           0       0.37      0.52      0.43        27
           1       0.66      0.51      0.57        49

    accuracy                           0.51        76
   macro avg       0.51      0.51      0.50        76
weighted avg       0.56      0.51      0.52        76

[[14 13]
 [24 25]]
KNN
Accuracy 0.5526315789473685
              precision    recall  f1-score   support

           0       0.41      0.63      0.50        27
           1       0.71      0.51      0.60        49

    accuracy                           0.55        76
   macro avg       0.56      0.57      0.55        76
weighted avg       0.61      0.55      0.56        76

[[17 10]
 [24 25]]
LogisticRegression
Accuracy 0.5263157894736842
              precision    recall  f1-score   support

           0       0.37      0.48      0.42        27
           1       0.66      0.55      0.60        49

    accuracy                     

/home/kush/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Xgboost
Accuracy 0.5263157894736842
              precision    recall  f1-score   support

           0       0.34      0.37      0.36        27
           1       0.64      0.61      0.62        49

    accuracy                           0.53        76
   macro avg       0.49      0.49      0.49        76
weighted avg       0.53      0.53      0.53        76

[[10 17]
 [19 30]]


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print(name)
    print("Accuracy", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

SVM
Accuracy 0.618421052631579
              precision    recall  f1-score   support

           0       0.33      0.07      0.12        27
           1       0.64      0.92      0.76        49

    accuracy                           0.62        76
   macro avg       0.49      0.50      0.44        76
weighted avg       0.53      0.62      0.53        76

[[ 2 25]
 [ 4 45]]
KNN
Accuracy 0.5131578947368421
              precision    recall  f1-score   support

           0       0.25      0.19      0.21        27
           1       0.61      0.69      0.65        49

    accuracy                           0.51        76
   macro avg       0.43      0.44      0.43        76
weighted avg       0.48      0.51      0.49        76

[[ 5 22]
 [15 34]]
LogisticRegression
Accuracy 0.5526315789473685
              precision    recall  f1-score   support

           0       0.27      0.15      0.19        27
           1       0.62      0.78      0.69        49

    accuracy                      